## Bach Functionality Demonstration

### 1. Demonstration of Free Flowing LSW

The following examples demonstrate the impact of forcings and the user allocation functionality for a single free draining LSW. The simulations are simulated with synthetic data to highlight the functionality of Bach, with the exception of example 1.9  and 1.10 which show the Bach prototype simulation of the Hupsel LSW.

- 1.1     No external forcing
- 1.2     Precipitation
- 1.3     Evaporation
- 1.4     Precipitation and evaporation
- 1.5     Infiltration
- 1.6     Urban Runoff
- 1.7     Allocation to a single user
- 1.8     Allocation to multiple users
- 1.9     Hupsel water balance (single LSW)
- 1.10    Hupsel water balance comparison with Mozart

### 2. Demonstration of a Level Controlled LSW

The following examples demonstrate the water management and user allocation functionality for a level controlled LSW. The simulations are simulated with synthetic data to highlight the functionality of Bach, with the exception of example 2.1 and 2.3 which show the Bach prototype simulation of the Tol LSW.

- 2.1     Tol water balance (single LSW)
- 2.2     Level Control and multiple users
- 2.3     Tol water balance comparison with Mozart.

In [ ]:
# Set up for demo simulations

# Load Julia Libraries
using Bach
using Duet
using Dates
using TOML
using Arrow
using DataFrames
import BasicModelInterface as BMI
using SciMLBase
using Graphs
using GLMakie
using CairoMakie
using Plots
using DataFrameMacros

In [ ]:

# Load TOML
config = TOML.parsefile("../run.toml");
# Load LSW Forcings Data
lswforcing = DataFrame(Arrow.Table("../data/input/6/forcing.arrow"));

#### 1.1 No external forcing

This fictional free flowing LSW has no input or output forcing flux. The LSW empties according to the Volume-Area_Discharge relationship.


In [ ]:
case = "emptying_sloping_profile"
## Code
## Set up
democonfig = Dict{String,Any}()
lsw_id = 1
democonfig["lsw_ids"] = [lsw_id]
democonfig["update_timestep"] = 86400.0
democonfig["starttime"] = Date("2022-01-01")
democonfig["endtime"] = Date("2022-02-01")
democonfig["state"] = DataFrame(location=lsw_id, volume=1e6)
democonfig["static"] = DataFrame(location=lsw_id, target_level=NaN, target_volume=NaN, depth_surface_water=NaN, local_surface_water_type='V')
democonfig["forcing"] = DataFrame(time=DateTime[], variable=Symbol[], location=Int[], value=Float64[])
democonfig["profile"] = DataFrame(location=lsw_id, volume=[0.0, 1e6], area=[1e6, 1e6], discharge=[0.0, 1e0], level=[10.0, 11.0])

## Simulate
reg = BMI.initialize(Bach.Register, democonfig)
solve!(reg.integrator);
# println(reg)

## Plot Results

fig_1a = Duet.plot_series(reg, lsw_id)
display(fig_1a)

### 1.2 Precipitation forcing

This fictional free flowing LSW is simulated with only the external forcing of synthetic precipitation data. The storage and the outflow respond to the preciptation as shown below.

In [ ]:
case = "precipitation"
## Code
## Set up
democonfig["starttime"] = Date("2019-01-01")
democonfig["endtime"] = Date("2020-01-01")
dummydata = @subset(lswforcing, :variable == Symbol("precipitation"), :location == 151358, :time >= config["starttime"], :time <= config["endtime"])
dummydata.location .= lsw_id
democonfig["forcing"] = DataFrame(time=dummydata.time, variable=dummydata.variable, location=dummydata.location, value=dummydata.value * 3) # Exaggerated for demo
democonfig["profile"] = DataFrame(location=lsw_id, volume=[0.0, 1e6], area=[1e6, 1e6], discharge=[0.0, 1e0], level=[10.0, 11.0])

# Simulate and Plot
reg = BMI.initialize(Bach.Register, democonfig)
solve!(reg.integrator)  # solve it until the end
# println(reg)

# Plot
fig_2 = Duet.plot_series(reg, lsw_id)
display(fig_2)


### 1.3 Evaporation forcing

The LSW loses water from evaporation. Outflow occurs according to the VAD as in example 1.1, but overall volume decline is faster rate due to additional loss from evaporation. Evaporation does not occur in an empty LSW.

The second example shows the LSW with no simulated discharge, so that the only output is evaporation

In [ ]:
case = "evaporation"
## Code
## Set up
lsw_id = 1
democonfig["starttime"] = Date("2019-01-01")
democonfig["endtime"] = Date("2019-06-01")
dummydata = @subset(lswforcing, :variable == Symbol("evaporation"), :location == 151358)
democonfig["forcing"] = DataFrame(time=dummydata.time, variable=dummydata.variable, location=lsw_id, value=dummydata.value * 3)
democonfig["profile"] = DataFrame(location=lsw_id, volume=[0.0, 1e6], area=[1e6, 1e6], discharge=[0.0, 1e0], level=[10.0, 11.0])

## Simulate
reg = BMI.initialize(Bach.Register, democonfig)
solve!(reg.integrator)  # solve it until the end
# println(reg)

## Plot
fig_3 = Duet.plot_series(reg, lsw_id)
display(fig_3)


In [ ]:

case = "evaporation2"
# Code
## Set up

config = Dict{String,Any}()
lsw_id = 1
config["lsw_ids"] = [lsw_id]
config["update_timestep"] = 86400.0
# config["saveat"] = 86400.0
starttime = DateTime("2019-01-01")
config["starttime"] = starttime
config["endtime"] = Date("2019-06-01")
config["state"] = DataFrame(location=lsw_id, volume=1e6)
config["static"] = DataFrame(location=lsw_id, target_level=NaN, target_volume=NaN,
    depth_surface_water=NaN, local_surface_water_type='V')
config["forcing"] = DataFrame(time=starttime, variable=:evaporation, location=lsw_id,
    value=1e-6)
config["profile"] = DataFrame(location=lsw_id, volume=[0.0, 1e6], area=[1e6, 1e6],
    discharge=[0.0, 0.0], level=[10.0, 11.0])

# Simulate
reg = BMI.initialize(Bach.Register, config)
solve!(reg.integrator)  # solve it until the end
println(reg)

# Plot
fig_s = Duet.plot_series(reg, lsw_id)
display(fig_s)

### 1.4 Evaporation and precipitation forcings

This example shows the evaporation and precipitation flux simulated together


In [ ]:
case = "evaporation_precipitation"
## Code
## Set up
democonfig["starttime"] = Date("2019-01-01")
democonfig["endtime"] = Date("2019-06-01")
dummydata_e = @subset(lswforcing, :variable == Symbol("evaporation"), :location == 151358)
dummydata_p = @subset(lswforcing, :variable == Symbol("precipitation"), :location == 151358)
dummydata = append!(dummydata_e, dummydata_p);
sort!(dummydata, [order(:time, rev=false)]);
democonfig["forcing"] = DataFrame(time=dummydata.time, variable=dummydata.variable, location=lsw_id, value=dummydata.value * 3)

## Simulate
reg = BMI.initialize(Bach.Register, democonfig)
solve!(reg.integrator)  # solve it until the end
# println(reg)

## Plot
fig_4 = Duet.plot_series(reg, lsw_id)
display(fig_4)

#### 1.5 Infiltration 

Infiltration is a negative flux of the LSW. This example shows the LSW storage responding to an enhanced forcing of infiltration. There is no outflow simulated of this LSW, so that the only output is the infiltration



In [ ]:
case = "Infiltration"
## Code
## Set up
config = Dict{String,Any}()
lsw_id = 1
config["lsw_ids"] = [lsw_id]
config["update_timestep"] = 86400.0
starttime = DateTime("2019-01-01")
config["starttime"] = starttime
config["endtime"] = Date("2019-02-01")
config["state"] = DataFrame(location=lsw_id, volume=1e6)
config["static"] = DataFrame(location=lsw_id, target_level=NaN, target_volume=NaN,
    depth_surface_water=NaN, local_surface_water_type='V')
config["forcing"] = DataFrame(time=starttime, variable=:infiltration,
    location=lsw_id, value=1.5e-6)
config["profile"] = DataFrame(location=lsw_id, volume=[0.0, 1e6], area=[1e6, 1e6],
    discharge=[0.0, 0.0], level=[10.0, 11.0])
## Simulate
reg = BMI.initialize(Bach.Register, config)
solve!(reg.integrator);  # solve it until the end

## Plot
fig_5 = Duet.plot_series(reg, lsw_id)
display(fig_5)

#### 1.6 Urban Runoff

Urban runoff is a surface water input to the LSW. This example shows the LSW storage responding to the influx forcing of urban runoff.

In [ ]:
case = "Urban Runoff"
## Code
## Set up
democonfig["starttime"] = Date("2019-01-01")
democonfig["endtime"] = Date("2019-06-01")
dummydata = @subset(lswforcing, :variable == Symbol("urban_runoff"), :location == 151358)
dummydata.value *= 3 # Emphasised to highlight the functionality
democonfig["forcing"] = DataFrame(time=dummydata.time, variable=dummydata.variable, location=lsw_id, value=dummydata.value * 3)

## Simulate
reg = BMI.initialize(Bach.Register, democonfig)
solve!(reg.integrator)  # solve it until the end
#println(reg)

## Plot
fig_6 = Duet.plot_series(reg, lsw_id)
display(fig_6)


### 1.7 Allocation to a single user (agriculture)

In this example, the Hupsel LSW is simulated with synthetic agricultural demand to demonstrate the Bach functionality to allocate to a General User

In [ ]:
case = "SingleUser"
## Code
## Set up
democonfig = TOML.parsefile("../run.toml");
lsw_id = 151358
democonfig["lsw_id"] = [lsw_id] # Hupsel
democonfig["starttime"] = Date("2019-01-01")
democonfig["endtime"] = Date("2019-06-01")
# Clear forcing data for agriculture and replace with synthetic data
dummydata = lswforcing
dummydata = @subset(dummydata :variable != Symbol("demand_agriculture") )
dummydata_demagric = @subset(lswforcing, :variable ==Symbol("demand_agriculture") ,value: = 0.2, :location == 131183)
dummydata = append!(dummydata, dummydata_demagric)
democonfig["forcing"] = DataFrame(time=dummydata.time, variable=dummydata.variable, location=lsw_id, value=dummydata.value)

## Simulate
reg = BMI.initialize(Bach.Register, democonfig)
solve!(reg.integrator);  # solve it until the end
println(reg)

## Plot
fig_6 = Duet.plot_series(reg, lsw_id)
display(fig_6)

# Duet.plot_genuser_series(reg, lsw_id)

### 1.8 Allocation to multiple users (agriculture and industry)

The allcoation is based upon demand and prioritisation of the users and the available water in the LSW. In a free flowing LSW only water from the LSW can be abstracted by the users: agriculture and industry.
In this example there are two users. Agriculture has higher prioirty than industry, therefore when there is a shortage of available water, agriculture abstracts water before industry as demonstrated.

In [ ]:
case = "MultipleUser"
## Set up
democonfig["starttime"] = Date("2019-01-01")
democonfig["endtime"] = Date("2019-06-01")
dummydata_dem_a.value .= 0.2 # To emphasise impact of abstraction
dummydata_prio_a = DataFrame(time=democonfig["starttime"], variable=Symbol("priority_industry"), location=lsw_id, value=1.0)
dummydata_dem_i = @subset(lswforcing, :variable == Symbol("demand_industry"), :location == 131183, :time >= democonfig["starttime"], :time <= democonfig["endtime"])
dummydata_dem_i.value *= dummydata_dem_a.value .* 0.5 # Industry demand at 50% of agriculture
dummydata_prio_i = DataFrame(time=democonfig["starttime"], variable=Symbol("priority_industry"), location=lsw_id, value=2.0)
dummydata = append!(dummydata_dem_a, dummydata_dem_i, dummydata_prio_a, dummydata_prio_i)
democonfig["forcing"] = DataFrame(time=dummydata.time, variable=dummydata.variable, location=lsw_id, value=dummydata.value)

## Simulate
reg = BMI.initialize(Bach.Register, democonfig)
solve!(reg.integrator)  # solve it until the end
# println(reg)

## Plot
# fig_7 = plot_genuser_series(reg, lsw_id, lswusers = ["agric", "indus"]) #need to update allusers function 
# display(fig_7)


### 1. 9 Water balance of a single LSW (Hupsel)

Simulation 1.9 is for the LSW Hupsel. The LSW is a free flowing LSW.


In [ ]:
## Code
## Set up
config = TOML.parsefile("run.toml")
config["lsw_id"] = [151358] # Hupsel. Disconnected

## Simulate
reg = BMI.initialize(Bach.Register, config)
println("solve! ", Time(now()))
@time solve!(reg.integrator)  # solve it until the end
println(reg)

# Plot
fig_9 = Duet.plot_series(reg, config["lsw_ids"][1]; level=true)
display(fig_9)

### 1.10 Water balance comparison Hupsel

The following two figures show Hupsel LSW water balance for the Bach prototype compared to the water balance simulated by Mozart, the precursor to Bach.
The figures show a good agreement between the two simulations

<img src="https://user-images.githubusercontent.com/4471859/179259174-0caccd4a-c51b-449e-873c-17d48cfc8870.png" alt="Mozart - Bach Water Balance Comparison" title="Mozart - Bach Water Balance Comparison" width="800"/>

<img src="https://user-images.githubusercontent.com/103200724/186892581-557e2a0a-7b17-47cc-b65c-a2f7e3e30b66.PNG" alt="Mozart - Bach Storage Comparison" title="Mozart - Bach Water Balance Comparison" width="800"/>


<img src="Mozart_comparison_Hupsel_2.PNG" alt="" title="Mozart - Bach Storage Comparison" width="800"/>

### 2.1 Single Level Controlled LSW (Tol)

Simulation 2.1 is for the LSW Tol. The LSW is a level controlled meaning that water is allocated to maintain water levels at a target level. This simulation demonstrates the LSW with the Bach prototype.

In [ ]:
## Code
case = "tol"
# Set Up
config = TOML.parsefile("run.toml")
config["lsw_id"] = [200164] # Tol
# Simulate
reg = BMI.initialize(Bach.Register, config)
println("solve! ", Time(now()))
@time solve!(reg.integrator)  # solve it until the end
println(reg)
# Plot
fig_12 = Duet.plot_series(reg, config["lsw_ids"][1]; level=true)
display(fig12)

# fig_13 = Duet.plot_wm_series(reg, config["lsw_ids"][1]; level = true)
# display(fig_13)


### 2.2 Single Level Controlled LSW with additional General Users

Simulation 2.2 is for a hypothetical level controlled LSW situation. This level control LSW has an additional users (agriculture) with a lower priority than the level control. In this scenario water is allocated to the users irder of priority, first based on the level of the LSW with respect to the target level, and additionally with water from the external network when required. The LSW is a level controlled meaning that water is allocated to maintain water levels at a target level. 

In [ ]:
case = "LevelControl_MultipleUsers"
## Set up
democonfig = TOML.parsefile("run.toml")
democonfig["lsw_id"] = [200164] # Take forcings from Tol
democonfig["starttime"] = Date("2019-01-01")
democonfig["endtime"] = Date("2019-06-01")
# Add in synthetic data for demand
forcings = lswforcing
# TODO create dummy data for water management and agric scenario to include

# democonfig["forcing"] = DataFrame(time=dummydata.time, variable=dummydata.variable, location=lsw_id, value=dummydata.value)

## Simulate
reg = BMI.initialize(Bach.Register, democonfig)
solve!(reg.integrator)  # solve it until the end
# println(reg)

## Plot
# fig_14 = plot_user_wm_series(reg, lsw_id, lswusers = ["agric", "indus"])
# display(fig_14)


### 2.3     Tol water balance comparison with Mozart

The following two figures show Tol LSW water balance for the Bach prototype compared to the water balance simulated by Mozart, the precursor to Bach.
The figures show a good agreement between the two simulations

<img src="https://user-images.githubusercontent.com/103200724/186892605-db217b55-13d6-48bb-a78e-3bbf4acc70c1.PNG" alt="Mozart - Bach Water Balance Comparison" title="Mozart - Bach Water Balance Comparison" width="800"/>

<img src="https://user-images.githubusercontent.com/103200724/186892628-7cb0d3cd-9439-46cb-b6b8-9908d9c7e09a.PNG" alt="Mozart - Bach Water Balance Comparison" title="Mozart - Bach Water Balance Comparison" width="800"/>

<img src="Mozart_comparison_Tol_2.PNG" alt="" title="Mozart - Bach Storage Comparison" width="800"/>